In [234]:
!ls

Client_Project_Ixchel.ipynb
Client_Project_John.ipynb
Client_Project_Kevin.ipynb
Ixchel_Twitter.ipynb
Weather
converting-text-files-to-combined-dictionary.ipynb
data
mycsv.csv
saved_tweets.csv
tweet
tweets_scrape_first_df.csv
twitter_credentials.json
working_df.csv


In [235]:
import pandas as pd

In [236]:
df = pd.read_csv('tweets_scrape_first_df.csv')

In [237]:
df.drop(columns='Unnamed: 0', inplace=True)

In [238]:
df.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
0,267716394043465728,2012-11-11 14:52:16,NaN,False,False,NaN,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy
1,298660085255770112,2013-02-05 00:11:27,NaN,False,False,NaN,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY
2,21860028236,2010-08-22 17:33:44,NaN,False,False,NaN,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1
3,1069021302667833344,2018-12-01 19:11:54,NaN,False,False,NaN,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale
4,767077182124486656,2016-08-20 15:13:46,NaN,False,False,NaN,3.0,0.0,7.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/767077182124486656,16145875.0,NotifyNYC


In [239]:
df.dtypes

ID                object
datetime          object
has_media         object
is_reply          object
is_retweet        object
medias            object
nbr_favorite     float64
nbr_reply        float64
nbr_retweet      float64
text              object
url               object
user_id          float64
usernameTweet     object
dtype: object

In [240]:
pd.to_datetime(df['datetime'][:12])

0    2012-11-11 14:52:16
1    2013-02-05 00:11:27
2    2010-08-22 17:33:44
3    2018-12-01 19:11:54
4    2016-08-20 15:13:46
5    2012-10-28 12:22:46
6    2018-03-02 22:08:23
7    2016-08-13 16:46:34
8    2016-08-13 16:38:28
9    2012-10-29 18:16:23
10   2012-10-31 04:19:06
11   2012-10-30 17:30:20
Name: datetime, dtype: datetime64[ns]

In [241]:
for i in df.datetime:
    if i[5:7] not in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        print(i)

20567184


In [242]:
df[df.datetime == '20567184']

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
2808,/Maggie4Obama/status/108170030474203136,20567184,Maggie4Obama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
df.iloc[2808]

ID               /Maggie4Obama/status/108170030474203136
datetime                                        20567184
has_media                                   Maggie4Obama
is_reply                                             NaN
is_retweet                                           NaN
medias                                               NaN
nbr_favorite                                         NaN
nbr_reply                                            NaN
nbr_retweet                                          NaN
text                                                 NaN
url                                                  NaN
user_id                                              NaN
usernameTweet                                        NaN
Name: 2808, dtype: object

In [244]:
df.drop(index=2808, inplace=True)

In [245]:
df[df.datetime == '20567184']

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet


In [246]:
df.datetime = pd.to_datetime(df.datetime)

In [247]:
df.dtypes

ID                       object
datetime         datetime64[ns]
has_media                object
is_reply                 object
is_retweet               object
medias                   object
nbr_favorite            float64
nbr_reply               float64
nbr_retweet             float64
text                     object
url                      object
user_id                 float64
usernameTweet            object
dtype: object

In [248]:
df.isnull().sum()

ID                  0
datetime            0
has_media        4225
is_reply            0
is_retweet          0
medias           4225
nbr_favorite        0
nbr_reply           0
nbr_retweet         0
text                0
url                 1
user_id             1
usernameTweet       1
dtype: int64

#### Unique Values

In [249]:
unique = []
for i in df.columns:
    uniques = {}
    uniques['column'] = i 
    uniques['unique values'] = df[i].nunique()
    unique.append(uniques)
pd.DataFrame(unique)

,column,unique values
0,ID,4374
1,datetime,4254
2,has_media,1
3,is_reply,2
4,is_retweet,1
5,medias,49
6,nbr_favorite,33
7,nbr_reply,19
8,nbr_retweet,45
9,text,4116


# Pre-Processing

### TweetTokenize and Lemmatize all of the Titles

In [250]:
# import tokenizer and stemmer
from nltk import word_tokenize, TweetTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [251]:
# define a function that takes in a tweet
# and tokenizes and stems that tweet
def stem_title(tweet):    
    stm = PorterStemmer()
    tokenized = word_tokenize(tweet)
    stem_tokes = []
    for toke in tokenized:
        stem_tokes.append(stm.stem(toke))
    stem_tokes

    comb = ''
    for stemmed in stem_tokes:
        comb += stemmed + ' '
    return comb

# define a function that takes in a tweet
# and tweet_tokenizes and lemmatizes that tweet
def lem_tweet(tweet):    
    lem = WordNetLemmatizer()
    tokenizer = TweetTokenizer()
    tokenized = tokenizer.tokenize(tweet)
    lemmed_tokes = []
    for toke in tokenized:
        lemmed_tokes.append(lem.lemmatize(toke))
    lemmed_tokes

    comb = ''
    for lemmed in lemmed_tokes:
        comb += lemmed + ' '
    return comb

In [252]:
# check that the function works
lem_tweet(df.text[3])

'about that power outage that is now over ... From @Verizon " The commercial power issue in your area may have been cleared . If you still have issue with your Verizon service , click here . " From @ConEdison <crickets> '

In [253]:
# create a new column by applying the function to the title column
# and check out the head to see if it worked
df['lemmed_tweet'] = df.text.apply(lem_tweet)
df.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet
0,267716394043465728,2012-11-11 14:52:16,NaN,False,False,NaN,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy,ConEd : NY Sandy power outage slip ; cost coul...
1,298660085255770112,2013-02-05 00:11:27,NaN,False,False,NaN,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY,@ ConEdison Power outage in queen
2,21860028236,2010-08-22 17:33:44,NaN,False,False,NaN,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1,Power outage in the Whitestone section of Quee...
3,1069021302667833344,2018-12-01 19:11:54,NaN,False,False,NaN,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale,about that power outage that is now over ... F...
4,767077182124486656,2016-08-20 15:13:46,NaN,False,False,NaN,3.0,0.0,7.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/767077182124486656,16145875.0,NotifyNYC,. @ConEdison is responding to a power outage i...


In [254]:
# check out the tail, too
df.tail()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet
4370,204532926447226880,2012-05-21 07:23:44,NaN,False,False,NaN,1.0,0.0,9.0,ConEd is responding to a power outage in ...,/NotifyNYC/status/204532926447226880,1.614588e+07,NotifyNYC,ConEd is responding to a power outage in SI zi...
4371,264374015714553858,2012-11-02 10:30:51,NaN,True,False,NaN,0.0,0.0,1.0,@ hhertzof Please check out our outage map ...,/ConEdison/status/264374015714553858,2.026208e+07,ConEdison,@ hhertzof Please check out our outage map at ...
4372,308557748671479808,2013-03-04 07:41:14,NaN,False,False,NaN,0.0,1.0,0.0,@ ConEdison Partial power outage in Green...,/curlycakes/status/308557748671479808,2.760942e+08,curlycakes,@ ConEdison Partial power outage in Greenwood ...
4373,969704423621648384,2018-03-02 17:42:05,NaN,False,False,NaN,0.0,1.0,0.0,"Just out of curiosity @EversourceCT , will the...",/yelenadasher/status/969704423621648384,2.791989e+09,yelenadasher,"Just out of curiosity @EversourceCT , will the..."
4374,951772242203070464,2018-01-12 06:06:00,NaN,False,False,NaN,1.0,0.0,4.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/951772242203070464,1.614588e+07,NotifyNYC,. @ConEdison is responding to a power outage i...


In [126]:
#df.to_csv('working_df.csv')

## Model Prep

### CountVectorize from Unlemmed

In [255]:
from sklearn.feature_extraction.text import CountVectorizer

In [256]:
# Instantiate
cv = CountVectorizer(stop_words='english', 
                     ngram_range=(1,1)
                    )

In [261]:
# Fit and Transform
sparse_df_unlemmed = cv.fit_transform(df.text)

In [264]:
dense_df_unlemmed = pd.DataFrame(sparse_df_unlemmed.todense(), columns=cv.get_feature_names())
dense_df.head()

,000,0075,03,032,10,100,101,10301,10302,10304,...,yesterday,yhoo,yonker,york,yorker,yorktown,youtu,zajp7i,zip,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [265]:
len(cv.get_feature_names())

7011

### CountVectorize from Lemmed

In [266]:
from sklearn.feature_extraction.text import CountVectorizer

In [267]:
# Instantiate
cv = CountVectorizer(stop_words='english', 
                     ngram_range=(1,1)
                    )

In [268]:
# Fit and Transform
sparse_df_lemmed = cv.fit_transform(df.lemmed_tweet)

In [269]:
dense_df_lemmed = pd.DataFrame(sparse_df_lemmed.todense(), columns=cv.get_feature_names())
dense_df.head()

,000,0075,03,032,10,100,101,10301,10302,10304,...,yesterday,yhoo,yonker,york,yorker,yorktown,youtu,zajp7i,zip,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [270]:
len(cv.get_feature_names())

6880

In [316]:
cv.get_feature_names()[6000:6008]

['superstormsandy',
 'supervisor',
 'supplier',
 'supply',
 'support',
 'supposed',
 'supposedly',
 'sure']

## Clustering

In [398]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
km = KMeans(n_clusters=3, random_state=42)

In [399]:
X_sc = ss.fit_transform(dense_df_lemmed)
km.fit(X_sc)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0)

In [400]:
from sklearn.metrics import silhouette_score

In [401]:
silhouette_score(X_sc, km.labels_)

0.5133360465681364

# Spacy Processing

In [405]:
import spacy
nlp = spacy.load('en')
doc1 = nlp(df.lemmed_tweet[2])
print(doc1)

Power outage in the Whitestone section of Queens . To report a loss of electric service , contact Con Edison at 1-800- 75 - CONED 


## Part-of-Speech Tagging

In [352]:
for token in doc:
    print('{} - {}'.format(token, token.pos_))

Power - NOUN
outage - NOUN
in - ADP
the - DET
Whitestone - PROPN
section - NOUN
of - ADP
Queens - PROPN
. - PUNCT
To - PART
report - VERB
a - DET
loss - NOUN
of - ADP
electric - ADJ
service - NOUN
, - PUNCT
contact - NOUN
Con - PROPN
Edison - PROPN
at - ADP
1 - NUM
- - PUNCT
800- - NUM
75 - NUM
- - PUNCT
CONED - NOUN


#### Extract Proper Nouns from Tweets

In [374]:
for tweet in df.lemmed_tweet[:30]:
    #print(nlp(i))
    doc = nlp(tweet)
    PROPN = []
    for token in doc:
        if token.pos_ == 'PROPN':
            PROPN.append(str(token))   
    print(PROPN)

['NY', 'Sandy', 'dlvr.it/2Sss2H']
['ConEdison', 'Power']
['Whitestone', 'Queens', 'Con', 'Edison']
['Verizon', '@ConEdison']
['@ConEdison', 'BX']
['ConEd', 'AQ', 'AAAA', '3ijeYFE', '~', 'Zup', '9nhRYBuHm1KS2tVqBGRDi67_ocWsu', 'Live', 'ConEd']
['ConEdison', 'Harrison', 'NY']
['NotifyNYC', 'ConEdison', 'SI', 'ZIP']
['@ConEdison', 'SI', 'ZIP']
['Handy', 'Sandy', 'Maps', 'cd37823361e17f570', 'ie', 'UTF', 'Current', 'ConEd', 'http']
['Manhattan', 'Power', 'Outages']
['Manhattan']
[]
['Queens', 'Sandy']
['ConEdison', 'Friday']
['Manor', 'Heights']
['ConEd', 'Power', 'Outage', 'Map', 'Hurricane', 'Sandy']
['RT', '@bbri1', '@ConEdison', 'Sts']
['NYC', 'ConEd', 'St', 'Manhattan', 'Report']
['BreakingNews', 'Power', 'Outage', 'Bronx', 'NY']
['North', 'East', 'Bronx', '@ConEdison', 'http://www']
['Bayside', 'QN', 'ZIP', 'ASL', 'http', ':/', 'bit.ly/1u2Z88W']
['Lower', '#', 'Manhattan', 'NYC', 'Hurricane', 'Sandy']
['@ConEdison']
['MT', '@NotifyNYC', 'Lighthouse', 'Hill', 'SI', 'ConEd', 'http://ww

## Noun-Phrase-Chunking

In [406]:
for chunk in doc1.noun_chunks:
    print('{} - {}'.format(chunk, chunk.label_))

Power outage - NP
the Whitestone section - NP
Queens - NP
a loss - NP
electric service - NP
- CONED - NP


#### Extract Noun Phrase Chunks from Tweets

In [377]:
for tweet in df.lemmed_tweet[:30]:
    #print(nlp(i))
    doc = nlp(tweet)
    NP = []
    for chunk in doc.noun_chunks:
        NP.append(str(chunk))   
    print(NP)

['NY Sandy power outage slip', 'cost', '/ dlvr.it/2Sss2H']
['ConEdison Power outage', 'queen']
['Power outage', 'the Whitestone section', 'Queens', 'a loss', 'electric service', '- CONED']
['that power outage', '@Verizon', 'The commercial power issue', 'your area', 'you', 'issue', 'your Verizon service', '@ConEdison']
['@ConEdison', 'a power outage', 'BX ZIPs', 'Report service loss', '1800-75- CONED']
['a power outage', 'you', 'ConEd', 'bckey', 'AQ', 'bctid', '| Live ConEd outage map', 'reference', '/ apps.coned.com/stormcenter_ex ternal / default.html …']
['ConEdison major power outage', 'Harrison NY']
['NotifyNYC', 'ConEdison', 'a power outage', 'SI ZIP', 'Report service loss', '75 - CONED', 'http', ':/ / on.nyc.gov/22lyehC']
['@ConEdison', 'a power outage', 'SI ZIP', 'Report service loss', '75 - CONED', 'http', ':/ / on.nyc.gov/22lyehC']
['Sandy Maps', 'ConEd power shutdown warning', '0.0004 cd37823361e17f570', 'ie', 'UTF', '… Current ConEd outage', '/ apps.coned.com/stormcenter_ex 

## Named-Entity-Recognition

In [407]:
for ent in doc1.ents:
    print(ent.text, '-', ent.label_)

Whitestone - GPE
Queens - ORG
Con Edison - PERSON
1 - CARDINAL


#### Extract Geopolitical Entity from Tweet

In [375]:
for tweet in df.lemmed_tweet[:30]:
    #print(nlp(i))
    doc = nlp(tweet)
    GPE = []
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            GPE.append(str(ent.text))   
    #print(doc)
    print(GPE)

[]
[]
['Whitestone']
['Verizon']
['BX']
[]
['Harrison NY']
[]
[]
[]
[]
['Manhattan']
[]
[]
[]
['Manor Heights']
[]
['RT']
['Manhattan']
[]
['Bronx']
['Bayside']
['Lower', 'Manhattan']
[]
['MT']
['Throggs Neck', 'BX']
['QN']
['NEW YORK']
['Williamsbridge', 'BX']
[]


### Dependency Parsing
Can we use this to get items like "Lighthouse Hill" that aren't recognized by Named-Entity-Recognition?

In [427]:
df.lemmed_tweet[24]

'MT @NotifyNYC Power outage in Lighthouse Hill section of SI . To report a loss of service ConEd 1-800-752-6633 or http://www . coned.com '

In [428]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(df.lemmed_tweet[24])
displacy.render(doc, style='dep')